# Dealing with marks: the turmite model

The [turmite model](https://en.wikipedia.org/wiki/Langton's_ant), developed by [Christopher Langton](https://en.wikipedia.org/wiki/Christopher_Langton) in 1986, is a very simple mono-agent model that exhibits an emergent behavior. It is based on 2 rules:

* If the turmite is on a patch that does not contain a mark, it turns right, drops a mark, and moves forward,

* If the turmite is on a patch that contains a mark, it turns left, removes the mark, and moves forward.

## Grab dependencies

In [1]:
#!ruby
#@repository("~/.m2/repository/")
#@dependency(group="fr.univ-artois.lgi2a", module="similar2logo-lib", version="1.0-SNAPSHOT")

java_import 'java.lang.Double'
java_import 'java.awt.geom.Point2D'
java_import 'fr.univ_artois.lgi2a.similar.extendedkernel.libs.abstractimpl.AbstractAgtDecisionModel'
java_import 'fr.univ_artois.lgi2a.similar.extendedkernel.simulationmodel.ISimulationParameters'
java_import 'fr.univ_artois.lgi2a.similar.microkernel.AgentCategory'
java_import 'fr.univ_artois.lgi2a.similar.microkernel.LevelIdentifier'
java_import 'fr.univ_artois.lgi2a.similar.microkernel.SimulationTimeStamp'
java_import 'fr.univ_artois.lgi2a.similar.microkernel.agents.IAgent4Engine'
java_import 'fr.univ_artois.lgi2a.similar.microkernel.agents.IGlobalState'
java_import 'fr.univ_artois.lgi2a.similar.microkernel.agents.ILocalStateOfAgent'
java_import 'fr.univ_artois.lgi2a.similar.microkernel.agents.IPerceivedData'
java_import 'fr.univ_artois.lgi2a.similar.microkernel.influences.InfluencesMap'
java_import 'fr.univ_artois.lgi2a.similar.microkernel.levels.ILevel'
java_import 'fr.univ_artois.lgi2a.similar2logo.kernel.initializations.AbstractLogoSimulationModel'
java_import 'fr.univ_artois.lgi2a.similar2logo.kernel.model.LogoSimulationParameters'
java_import 'fr.univ_artois.lgi2a.similar2logo.kernel.model.agents.turtle.TurtleAgentCategory'
java_import 'fr.univ_artois.lgi2a.similar2logo.kernel.model.agents.turtle.TurtleFactory'
java_import 'fr.univ_artois.lgi2a.similar2logo.kernel.model.environment.LogoEnvPLS'
java_import 'fr.univ_artois.lgi2a.similar2logo.kernel.model.environment.Mark'
java_import 'fr.univ_artois.lgi2a.similar2logo.kernel.model.influences.ChangeDirection'
java_import 'fr.univ_artois.lgi2a.similar2logo.kernel.model.influences.DropMark'
java_import 'fr.univ_artois.lgi2a.similar2logo.kernel.model.influences.RemoveMark'
java_import 'fr.univ_artois.lgi2a.similar2logo.kernel.model.levels.LogoSimulationLevelList'
java_import 'fr.univ_artois.lgi2a.similar2logo.lib.model.ConeBasedPerceptionModel'
java_import 'fr.univ_artois.lgi2a.similar2logo.lib.probes.LogoRealTimeMatcher'
java_import 'fr.univ_artois.lgi2a.similar2logo.lib.tools.html.Similar2LogoWebRunner'

Resolving dependency: fr.univ_artois.lgi2a#similar2logo-lib;1.0-SNAPSHOT {default=[default]}
Preparing to download artifact fr.univ_artois.lgi2a#similar2logo-lib;1.0-SNAPSHOT!similar2logo-lib.jar
Preparing to download artifact fr.univ_artois.lgi2a#similar2logo-kernel;1.0-SNAPSHOT!similar2logo-kernel.jar
Preparing to download artifact com.sparkjava#spark-core;2.7.2!spark-core.jar(bundle)
Preparing to download artifact org.slf4j#slf4j-simple;1.7.13!slf4j-simple.jar
Preparing to download artifact org.apache.commons#commons-math3;3.6.1!commons-math3.jar
Preparing to download artifact fr.univ_artois.lgi2a#similar-microKernel;1.0-SNAPSHOT!similar-microKernel.jar
Preparing to download artifact fr.univ_artois.lgi2a#similar-microKernel-commonLibs;1.0-SNAPSHOT!similar-microKernel-commonLibs.jar
Preparing to download artifact fr.univ_artois.lgi2a#similar-extendedKernel;1.0-SNAPSHOT!similar-extendedKernel.jar
Preparing to download artifact fr.univ_artois.lgi2a#similar-extendedKernel-extendedLibs;1

Java::FrLgi2aSimilar2logoLibToolsHtml::Similar2LogoHtmlRunner


## The decision model

The decision model implements the above described rules :

In [2]:
class TurmiteDecisionModel < AbstractAgtDecisionModel
  
  def initialize
    super(LogoSimulationLevelList::LOGO)
  end
  
  def decide(
      timeLowerBound,
      timeUpperBound,
      globalState,
      publicLocalState,
      privateLocalState,
      perceivedData,
      producedInfluences
    )
    if perceivedData.getMarks.empty?
      producedInfluences.add(
        ChangeDirection.new(
          timeLowerBound,
          timeUpperBound,
          Math::PI/2,
          publicLocalState
        )
      )
      producedInfluences.add(
        DropMark.new(
          timeLowerBound,
          timeUpperBound,
          Mark.new(
            publicLocalState.getLocation.clone,
            nil
          )
        )
      )
    else
      producedInfluences.add(
        ChangeDirection.new(
          timeLowerBound,
          timeUpperBound,
          -Math::PI/2,
          publicLocalState
        )
      )
      producedInfluences.add(
        RemoveMark.new(
          timeLowerBound,
          timeUpperBound,
          perceivedData.getMarks.iterator.next.getContent
        )
      )
    end
  end
end

No Outputs

## The simulation model

The simulation model generates a turmite heading north at the location 10.5,10.5 with a speed of 1 and an acceleration of 0:


In [3]:
class TurmiteSimulationModel < AbstractLogoSimulationModel
  def generateAgents(p, levels)
      result =  AgentInitializationData.new
      turtle = TurtleFactory::generate(
        ConeBasedPerceptionModel.new(0, 2*Math::PI, false, true, false),
        TurmiteDecisionModel.new,
        AgentCategory.new("turmite", TurtleAgentCategory::CATEGORY),
        LogoEnvPLS::NORTH,
        1,
        0,
        10.5,
        10.5
      )
      result.agents.add(turtle)
      return result
    end
end

No Outputs

## Launch the HTML runner

In this case we want to observe turtles and marks.

In [5]:
runner = Similar2LogoHtmlRunner.new
runner.config.setExportAgents(true)
runner.config.setExportMarks( true )
runner.initializeRunner(TurmiteSimulationModel.new(LogoSimulationParameters.new))
runner.showView
runner.addProbe("Real time matcher", LogoRealTimeMatcher.new(20))

No Outputs